In [1]:
# We need PyTorch and GPyTorch
import torch
from torch import Tensor
import gpytorch
# Gaussian processes really don't work well without 64-bit precision
torch.set_default_dtype(torch.float64)
# For reproducibility:
torch.random.manual_seed(12345)
import sys
sys.path.append("..")

from norm_constrain import Norm2ConstrainedContainer_SE, Norm2ConstrainedContainer_rational, Norm2ConstrainedContainer_ConvexCombination, Norm2ConstrainedContainer_KroneckerProduct

# And a few more players
import numpy as np
from scipy.integrate import quad
from matplotlib import pyplot as plt
import matplotlib.cm as cm
%matplotlib inline

In [2]:
# Import the data, check normalization, which should be \pi/2

with open('wgn.VMC_1.nucl.he4', 'r') as fd:
    lines = fd.readlines()
    
nk = 100 ; nr = 201
x = [] ; y = [] ; err = []
for line in lines[3:]:
    buf = line.split()
    x.append([float(buf[1]),float(buf[0])])
    y.append(float(buf[2]))
    err.append(float(buf[3]))
x = np.array(x) ; y = np.array(y) ; err = np.array(err)
xx = x.reshape((nr,nk,-1)) ; yy=y.reshape((nr,nk)) ; errr=err.reshape((nr,nk))
print(xx.shape)
print(yy.shape)
dr = xx[1,0,0] - xx[0,0,0] ; dk = xx[0,2,1] - xx[0,1,1] ; delta = np.array([dr, dk])
print(delta)

xx = xx[1:,1:,:] - delta/2 # Cell centers
br = (yy[1:,1:] - yy[:-1,1:])/dr ; bk = (yy[1:,1:] - yy[1:,:-1])/dk
beta = np.array([br,bk]).transpose((1,2,0))
print(beta.shape)
yy = yy[1:,1:] - 0.5 * beta.dot(delta) # Interpolation to cell centers

ig = (yy * xx[:,:,0]**2 * xx[:,:,1]**2).sum() *dk*dr *2 / np.pi
print("Integral / (pi/2) = %f"%ig)


(201, 100, 2)
(201, 100)
[0.025 0.08 ]
(200, 99, 2)
Integral / (pi/2) = 1.006803


In [3]:
# Subsample training data
subrate = 5 # per dimension, full subsampling rate is square of this
train_x = torch.tensor(xx[::subrate,::subrate,:]).reshape(-1,2)
train_y = torch.tensor(yy[::subrate,::subrate]).flatten()
train_err = torch.tensor(errr[::subrate,::subrate]).flatten()**2
print(train_x.shape)

# Normalization value
nval = np.pi/2


torch.Size([800, 2])


In [12]:
# Create the GPyTorch model
class GPRegressionModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, nval):
        super(GPRegressionModel, self).__init__(train_x, train_y, likelihood)

        self.nm0_0 = Norm2ConstrainedContainer_SE()
        self.nm0_0.sigma = 0.292 ; self.nm0_0.gamma = 3.432 ; self.nm0_0.A = 0.426
        self.nm0_1 = Norm2ConstrainedContainer_SE(fix_amplitude=1.0)
        self.nm0_1.sigma = 0.919 ; self.nm0_1.gamma = 2.379
        kernels = [self.nm0_0,self.nm0_1]
        self.nm0 = Norm2ConstrainedContainer_KroneckerProduct(kernels, norm_val=nval)

        self.covar_module = self.nm0.covar_module
        self.mean_module = self.nm0.mean_module

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

likelihood = gpytorch.likelihoods.FixedNoiseGaussianLikelihood(noise=train_err)

model = GPRegressionModel(train_x, train_y, likelihood, nval)

/home/carlo/python_venvs/pytorch.venv/lib/python3.10/site-packages/gpytorch/likelihoods/noise_models.py:148: NumericalWarning: Very small noise values detected. This will likely lead to numerical instabilities. Rounding small noise values up to 1e-06.
  warnings.warn(


In [13]:
# Training
training_iter = 500

model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

for i in range(training_iter):
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    # Output from model
    output = model(train_x)
    # Calc loss and backprop gradients
    loss = -mll(output, train_y)
    loss.backward()
    print('Iter %d/%d - Loss: %.3f\nSE_k: sigma: %.3f  gamma: %.3f  A: %.3f\nSE_r: sigma: %.3f  gamma: %.3f' % (
        i + 1, training_iter, loss.item(),
        model.nm0.kernels[0].sigma.item(),
        model.nm0.kernels[0].gamma.item(),
        model.nm0.kernels[0].A.item(),
        model.nm0.kernels[1].sigma.item(),
        model.nm0.kernels[1].gamma.item(),
    ))
    optimizer.step()

/home/carlo/python_venvs/pytorch.venv/lib/python3.10/site-packages/gpytorch/likelihoods/gaussian_likelihood.py:300: GPInputWarning: You have passed data through a FixedNoiseGaussianLikelihood that did not match the size of the fixed noise, *and* you did not specify noise. This is treated as a no-op.
  warnings.warn(


Iter 1/500 - Loss: -5.917
SE_k: sigma: 0.292  gamma: 3.432  A: 0.426
SE_r: sigma: 0.919  gamma: 2.379
Iter 2/500 - Loss: -5.367
SE_k: sigma: 0.268  gamma: 3.529  A: 0.462
SE_r: sigma: 0.860  gamma: 2.470
Iter 3/500 - Loss: -5.843
SE_k: sigma: 0.285  gamma: 3.458  A: 0.435
SE_r: sigma: 0.903  gamma: 2.403
Iter 4/500 - Loss: -4.796
SE_k: sigma: 0.306  gamma: 3.378  A: 0.407
SE_r: sigma: 0.953  gamma: 2.328
Iter 5/500 - Loss: -5.907
SE_k: sigma: 0.294  gamma: 3.424  A: 0.423
SE_r: sigma: 0.924  gamma: 2.372
Iter 6/500 - Loss: -5.798
SE_k: sigma: 0.283  gamma: 3.465  A: 0.437
SE_r: sigma: 0.900  gamma: 2.411
Iter 7/500 - Loss: -5.594
SE_k: sigma: 0.275  gamma: 3.496  A: 0.448
SE_r: sigma: 0.882  gamma: 2.441
Iter 8/500 - Loss: -5.437
SE_k: sigma: 0.269  gamma: 3.517  A: 0.455
SE_r: sigma: 0.870  gamma: 2.462
Iter 9/500 - Loss: -5.338
SE_k: sigma: 0.265  gamma: 3.529  A: 0.458
SE_r: sigma: 0.864  gamma: 2.475
Iter 10/500 - Loss: -5.290
SE_k: sigma: 0.263  gamma: 3.534  A: 0.459
SE_r: sigma:

In [14]:
print("Final State:")
print('Iter %d/%d - Loss: %.3f\nSE_k: sigma: %.3f  gamma: %.3f  A: %.3f\nSE_r: sigma: %.3f  gamma: %.3f' % (
        i + 1, training_iter, loss.item(),
        model.nm0.kernels[0].sigma.item(),
        model.nm0.kernels[0].gamma.item(),
        model.nm0.kernels[0].A.item(),
        model.nm0.kernels[1].sigma.item(),
        model.nm0.kernels[1].gamma.item(),
    ))

Final State:
Iter 500/500 - Loss: -5.917
SE_k: sigma: 0.292  gamma: 3.432  A: 0.426
SE_r: sigma: 0.919  gamma: 2.379
